![logoAEMET](img/logoAEMET_05.png)

- **Autor:** Jose Luis Casado
- **Institución:** AEMET

---

# Descarga de datos con MARS

---

<div class="alert alert-info"><b>Nota:</b> Algunos de los enlaces puestos aquí solo funcionarán si nos hemos registrado con usuario y contraseña del ECMWF</div>

MARS (Meteorological Archival and Retrieval System) es la base de datos del ECMWF.  
Tiene un gran tamaño ([378 petabytes en octubre 2022](https://www.ecmwf.int/en/about/media-centre/key-facts-and-figures#:~:text=As%20of%20October%202022%2C%20it,fields%20are%20stored%20in%20MARS)), por lo que es conveniente conocer la mejor forma de extraer información de ella.

Se puede encontrar la [documentación de MARS](https://confluence.ecmwf.int/display/UDOC/MARS+user+documentation) en este enlace, la página principal desde la cual se puede acceder a todos los detalles. En esta presentación seguiremos a grandes rasgos esta documentación.

Para conectarse a los servidores del ECMWF para realizar consultas hay que usar [teleport y ssh](https://confluence.ecmwf.int/display/UDOC/Teleport+SSH+Access)   
También se puede utilizar una API desarrollada por el ECMWF, aunque tiene menos capacidades (se explicará en una presentación posterior).

### Exploración de MARS con la web del ECMWF

El objeto más pequeño al que nos podemos referir en MARS es el campo meteorológico (un "mapa" de un parámetro). Esto implica que obtener una serie temporal en un punto determinado puede ser algo costoso, ya que requiere descargar los "mapas" completos para cada instante de tiempo.

La arquitectura de MARS se explica en más detalle en esta [página web](https://confluence.ecmwf.int/display/UDOC/MARS+architecture) y en este [documento](https://confluence.ecmwf.int/download/attachments/45754389/server_architecture.pdf?version=1&modificationDate=1431093939498&api=v2).

Para acceder a MARS con la web del ECMWF podemos llegar a través de su [página de predicciones general](https://www.ecmwf.int/en/forecasts) en el apartado Datasets, o directamente a través de la [página del catálogo de MARS](https://www.ecmwf.int/en/forecasts/datasets/search?f%5B0%5D=sm_field_dataset_urls_filter%3Afield_dataset_mars_url).  

Aunque es posible realizar peticiones a MARS a través de la web, no es aconsejable. Sin embargo sí es útil que la propia página muestre la sintaxis correcta a usar en una petición, y también para ver si hay datos disponibles, o el coste de descargarlos.

La web también muestra el despliegue del [árbol de MARS](https://confluence.ecmwf.int/download/attachments/77223119/Screenshot_20170605_082349.png?version=1&modificationDate=1496647487359&api=v2) al ir seleccionando cada una de las keywords, algo que es importante tener en cuenta a la hora de hacer peticiones a MARS de forma óptima.

### Sintaxis de MARS. Palabras clave (keywords) más importantes

En la documentación en línea se puede encontrar información de [la sintaxis de MARS](https://confluence.ecmwf.int/display/UDOC/MARS+command+and+request+syntax), así como [detalles sobre cada una de las keywords](https://confluence.ecmwf.int/display/UDOC/Keywords+in+MARS+and+Dissemination+requests) que se pueden utilizar en una consulta a MARS.


Este es el formato de una consulta típica a MARS:

```
mars << END  
retrieve,  
class=od,  
stream=oper,  
expver=1,  
type=fc,  
levtype=sfc,  
param=167.128/169.128/246.228/247.228,  
date=20210108,  
time=00:00:00,  
step=6/12/18/24,  
repres=ll,  
grib=0.1/0.1,  
area=44.0/-10.0/35.0/5.0,  
use=infrequent,  
target=$SCRATCH/ecmwf_surface.grib
END  
```

Explicación de algunas de las keywords más importantes (no tienen por qué seguir este orden):
- retrieve/list/read/compute: comando a utilizar. Normalmente usaremos siempre retrieve (o list)
- class: clase de dato archivado (operacional, reanálisis...)
- stream: sistema de predicción (determinista, ensemble, oleaje...)
- type: tipo de campo (predicción, análisis, miembro de control en un ensemble...)
- levtype: tipo de nivel (superficie, presión, modelo...)
- param: parámetro/s a descargar. En este caso estamos solicitando la temperatura a 2m (167.128), la radiación global (169.128), y el viento a 100m (246.228 y 247.228)
- date: fecha/s de la descarga
- time: "pasada" o hora de ejecución (run) del modelo
- step: alcance o horas de predicción
- repres: representación del dato archivado (ll si es longitud-latitud)
- grid: resolución (longitud/latitud si la repres es ll)
- area: región que queremos recortar (Norte/Oeste/Sur/Este)
- target: fichero de salida

El separador "/" permite dar varios valores a una keyword. Además las keywords *to* y *by* permiten dan un rango de valores (por ejemplo 3/to/24/by/3).

Notar que el modelo está archivado en una [malla gaussiana reducida](https://confluence.ecmwf.int/display/FCST/Gaussian+grids), no en la longitud-latitud, más cómoda, que normalmente utilizamos en la práctica. Por ello MARS al descargar los datos interpola a la malla longitud-latitud que le especifiquemos. Para ello usa la [librería de interpolación MIR](https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR) internamente.

Para saber qué código numérico corresponde a cada parámetro podemos consultar las [tablas de parámetros del ECMWF](https://codes.ecmwf.int/grib/param-db/). Las dos que más a menudo utilizaremos (si escogemos el formato grib1) son la 128 (tabla standard) y la 228 (tabla standard2).

### List y retrieve. Acceso a disco y cinta

Al ejecutar una petición es conveniente hacerlo en modo batch, usando el [planificador de procesos SLURM](https://confluence.ecmwf.int/display/UDOC/HPC2020%3A+Batch+system), que se encuentra instalado en el servidor. Nosotros normalmente solo necesitaremos utilizar algunos comandos básicos (principalmente sbatch para lanzar las peticiones).

Antes de utilizar el comando `retrieve` para descargar los datos es recomendable comprobar el coste computacional que esta petición va a tener. Para ello hay que usar el comando `list`, incluyendo la keyword *output* con el valor *cost* (observar que con `list` necesitamos menos keywords):

```
list,  
class=od,  
stream=oper,  
expver=1,  
type=fc,  
levtype=sfc,  
param=167.128/169.128/246.228/247.228,  
date=20210108,  
time=00:00:00,  
step=6/12/18/24,  
output=cost,
target="./list_surface.txt"  
```

El fichero de log (o el fichero que le indiquemos con *target*) nos dará información sobre la existencia de los datos, sobre su cantidad y dónde están almacenados:

*size=211273408;  
number_of_fields=16;  
online_size=211273408;  
off_line_size=0;  
number_of_tape_files=0;  
number_of_disk_files=3;  
number_of_online_fields=16;  
number_of_offline_fields=0;  
number_of_tapes=0;*

Los datos que están en disco se extraerán muy rápidamente. Los datos que están en cinta pueden tardar mucho en descargarse (¡sobre todo si alguien ya está utilizando esa cinta o esas cintas).

La consulta puede dar error si hay un error de sintaxis, o si sobrepasamos ciertos límites establecidos (por ejemplo el número de campos solicitados).

Una vez que hemos comprobado que nuestra petición es adecuada, o la hemos modificado para que sea más óptima, podemos ejecutarla con `retrieve`:

```
retrieve,  
class=od,  
stream=oper,  
expver=1,  
type=fc,  
levtype=sfc,  
param=167.128/246.228/247.228,  
date=20210108,  
time=00:00:00,  
step=6/12/18/24,  
repres=ll,  
grib=0.1/0.1,  
area=44.0/-10.0/35.0/5.0,  
use=infrequent,  
target="$SCRATCH/ecmwf_surface.grib"  
```

Notar que hemos incluido la keyword *use* para que los datos no se guarden en caché.

Aquí hay otros dos ejemplos. El primero devuelve predicciones de temperatura en el nivel de presión de 850 mb (observar que hay que utilizar una nueva keyword, *levelist*):
    
```
retrieve,  
class=od,  
stream=oper,  
expver=1,  
type=fc,  
levtype=pl,  
levelist=850,  
param=130.128,  
date=20210108,  
time=00:00:00,  
step=6/12/18/24,  
repres=ll,  
grib=0.1/0.1,  
area=44.0/-10.0/35.0/5.0,  
use=infrequent,  
target="$SCRATCH/ecmwf_levels.grib"  
```

El segundo devuelve datos de viento a 100m de ERA5:

```
retrieve,  
class=ea,  
stream=oper,  
expver=1,  
type=an,  
date=20210108,  
time=00:00:00/06:00:00/12:00:00/18:00:00,  
levtype=sfc,  
param=246.228/247.228,  
repres=ll,  
grib=0.25/0.25,  
area=44.0/-10.0/35.0/5.0,  
target="$SCRATCH/ecmwf_reanalysis.grib"  
```

Después de que MARS descargue los datos en el servidor del ECMWF, ya se pueden transferir por medio de ftp a nuestro ordenador local. También se puede programar su transferencia automática por medio de la aplicación [ectrans](https://confluence.ecmwf.int/display/ECAC/Unattended+file+transfer+-+ectrans), proporcionada por el ECMWF.

Es buena idea ver la [actividad en MARS](https://apps.ecmwf.int/mars-activity/) para comprobar el estado de nuestra petición, y si la base de datos está muy ocupada con peticiones (algo que nos puede afectar indirectamente).

### Buenas prácticas

El ECMWF recomienda seguir varias [buenas prácticas](https://confluence.ecmwf.int/display/UDOC/Operational+Data%3A+Guidelines+to+write+efficient+MARS+requests) que pueden optimizar las consultas, haciéndolas más eficientes:

- Si hay que descargar una gran cantidad de datos, analizar primero en detalle la mejor forma de hacerlo.
- Utilizar el comando list para ver previamente qué recursos se van a usar.
- Intentar descargar todos los datos posibles en una sola petición, siempre que estén en la misma cinta (en particular, todos los parámetros, niveles y steps deberían ir juntos en la misma petición).
- Si la petición es muy grande y requiere de muchas cintas, dividirla en varias más pequeñas, pero teniendo en cuenta la arquitectura de MARS para optimizarlas, de forma que cada cinta se lea solo una vez.
- Preparar las peticiones intentando prever posibles embotellamientos de MARS.